<a href="https://colab.research.google.com/github/C23-PS210-Londri-App/ML-Repository/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install PySastrawi
import tensorflow as tf
import pandas as pd
import numpy as np
import json

from google.colab import drive
from sklearn.feature_extraction.text import CountVectorizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tensorflow.keras.layers import Embedding, Dense, Input, Concatenate, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [7]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
file_path = '/content/drive/MyDrive/capstone_project/laundry_dataset.csv'

df = pd.read_csv(file_path)
df

,link,title,category,address,open_hours,popular_times,website,phone,plus_code,review_count,...,price_range,data_id,images,reservations,order_online,menu,owner,complete_address,about,user_reviews
0,https://www.google.com/maps/place/Lavanderia/d...,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...","{""Friday"":[""7:30 AM–9 PM""],""Monday"":[""7:30 AM–...","{""Friday"":{""10"":50,""11"":50,""12"":31,""13"":43,""14...",NaN,NaN,"RCVR+Q3 Masale, Makassar City, South Sulawesi",95,...,NaN,0x2dbee2bf4e64e56f:0xf87e6d7309d11f1,"[{""title"":""All"",""image"":""https://lh5.googleuse...",NaN,NaN,"{""link"":"""",""source"":""""}","{""id"":""103816956691432943282"",""name"":""Lavander...","{""borough"":""Masale, Panakkukang"",""street"":""Jl....","[{""id"":""service_options"",""name"":""Service optio...","[{""Name"":""Ade Lhinglhung"",""ProfilePicture"":""ht..."
1,https://www.google.com/maps/place/LaundrySaja/...,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...","{""Friday"":[""7 AM–9 PM""],""Monday"":[""7 AM–9 PM""]...","{""Friday"":{""10"":15,""11"":0,""12"":0,""13"":0,""14"":0...",https://www.instagram.com/laundrysaja_id/,0853-9755-6884,"RCQ7+H6 Mariso, Makassar City, South Sulawesi",9,...,NaN,0x2dbf1dd8c0928c55:0x458ab705d5e9ef46,"[{""title"":""All"",""image"":""https://lh5.googleuse...",NaN,NaN,"{""link"":"""",""source"":""""}","{""id"":""110461900888172896954"",""name"":""LaundryS...","{""borough"":""Mariso"",""street"":""Jl. Kakatua No.3...","[{""id"":""service_options"",""name"":""Service optio...","[{""Name"":""Refly Nakcoc"",""ProfilePicture"":""http..."
2,https://www.google.com/maps/place/Zap+Laundry/...,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...","{""Friday"":[""7 AM–9 PM""],""Monday"":[""7 AM–9 PM""]...","{""Friday"":{""10"":75,""11"":61,""12"":42,""13"":37,""14...",http://www.zaplaundry.com/,0878-6999-6679,"VF5Q+7J Tamalanrea Jaya, Makassar City, South ...",140,...,NaN,0x2dbee34a5d0d5a6b:0x876d2bed78883721,"[{""title"":""All"",""image"":""https://lh5.googleuse...",NaN,NaN,"{""link"":"""",""source"":""""}","{""id"":""104773887339071515463"",""name"":""Zap Laun...","{""borough"":""Tamalanrea Jaya, Tamalanrea"",""stre...","[{""id"":""accessibility"",""name"":""Accessibility"",...","[{""Name"":""Triwiyanti Triwiyanti"",""ProfilePictu..."
3,https://www.google.com/maps/place/QnC+Laundry+...,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...","{""Friday"":[""7 AM–12 PM"",""1–10 PM""],""Monday"":[""...","{""Friday"":{""10"":33,""11"":43,""12"":0,""13"":31,""14""...",https://www.qnclaundry.net/,0882-0224-14173,"RCMX+8J Kassi-Kassi, Makassar City, South Sula...",598,...,NaN,0x2dbee39f4bb2b275:0xaad8234ed3dd702b,"[{""title"":""All"",""image"":""https://lh5.googleuse...",NaN,"[{""link"":""https://api.whatsapp.com/send/?phone...","{""link"":"""",""source"":""""}","{""id"":""110421932829577435298"",""name"":""QnC Laun...","{""borough"":""Kassi-Kassi, Rappocini"",""street"":""...","[{""id"":""service_options"",""name"":""Service optio...","[{""Name"":""isra stimiyapmi"",""ProfilePicture"":""h..."
4,https://www.google.com/maps/place/Laundromad+M...,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...","{""Friday"":[""8 AM–11 PM""],""Monday"":[""8 AM–11 PM...","{""Friday"":{""10"":26,""11"":26,""12"":26,""13"":42,""14...",NaN,0852-5601-0299,"VF9X+7X Tamalanrea, Makassar City, South Sulawesi",75,...,NaN,0x2dbefcae31bb3e63:0xa51a7ff56c9d570e,"[{""title"":""All"",""image"":""https://lh5.googleuse...",NaN,NaN,"{""link"":"""",""source"":""""}","{""id"":""103237755937557818310"",""name"":""Laundrom...","{""borough"":""Tamalanrea"",""street"":""Jl. Bumi Tam...","[{""id"":""accessibility"",""name"":""Accessibility"",...","[{""Name"":""andi adinda"",""ProfilePicture"":""https..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,https://www.google.

In [9]:
df.columns

Index(['link', 'title', 'category', 'address', 'open_hours', 'popular_times',
       'website', 'phone', 'plus_code', 'review_count', 'review_rating',
       'reviews_per_rating', 'latitude', 'longitude', 'cid', 'status',
       'descriptions', 'reviews_link', 'thumbnail', 'timezone', 'price_range',
       'data_id', 'images', 'reservations', 'order_online', 'menu', 'owner',
       'complete_address', 'about', 'user_reviews'],
      dtype='object')

In [10]:
selected_columns = ['title','category','address','open_hours','review_count','review_rating','reviews_per_rating','latitude','longitude','user_reviews']
df = df[selected_columns]
df

,title,category,address,open_hours,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews
0,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...","{""Friday"":[""7:30 AM–9 PM""],""Monday"":[""7:30 AM–...",95,4.900.000,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155.502,119.440.242,"[{""Name"":""Ade Lhinglhung"",""ProfilePicture"":""ht..."
1,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...","{""Friday"":[""7 AM–9 PM""],""Monday"":[""7 AM–9 PM""]...",9,4.600.000,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161.116,119.413.019,"[{""Name"":""Refly Nakcoc"",""ProfilePicture"":""http..."
2,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...","{""Friday"":[""7 AM–9 PM""],""Monday"":[""7 AM–9 PM""]...",140,4.300.000,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141.826,119.489.119,"[{""Name"":""Triwiyanti Triwiyanti"",""ProfilePictu..."
3,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...","{""Friday"":[""7 AM–12 PM"",""1–10 PM""],""Monday"":[""...",598,4.900.000,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166.727,119.449.074,"[{""Name"":""isra stimiyapmi"",""ProfilePicture"":""h..."
4,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...","{""Friday"":[""8 AM–11 PM""],""Monday"":[""8 AM–11 PM...",75,4.500.000,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131.841,119.499.985,"[{""Name"":""andi adinda"",""ProfilePicture"":""https..."
...,...,...,...,...,...,...,...,...,...,...
808,Laundry,Laundry,"HPG3+W8G Laundry, Jl. Arief Rahman Hakim, Suka...",{},0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.577.300,98.703.334,NaN
809,Laundry dry & clean,Laundry,"Jl. Jamin Ginting No.96, Mangga, Kec. Medan Tu...","{""Friday"":[""8 AM–10 PM""],""Monday"":[""8 AM–10 PM...",0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.528.993,98.637.205,NaN
810,Michelle Laundry & Dry Clean,Laundry,"JM8X+532 Michelle Laundry & Dry Clean, Jl. Tua...","{""Friday"":[""8 AM–10 PM""],""Monday"":[""8 AM–10 PM...",1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615.383,98.697.644,"[{""Name"":""Eben RHS"",""ProfilePicture"":""https://..."
811,Laundry,Janitorial service,"HPV6+HX6 Laundry, Gg. Permata, Bantan, Kec. Me...",{},1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.593.911,98.712.397,"[{""Name"":""Aditya Masni"",""ProfilePicture"":""http..."


In [11]:
df_no_duplicates = df.drop_duplicates()

In [12]:
df_no_duplicates.duplicated().sum()

0

In [13]:
df.isna().sum()

title                   0
category                3
address                 1
open_hours              0
review_count            0
review_rating           0
reviews_per_rating      0
latitude                0
longitude               0
user_reviews          121
dtype: int64

In [14]:
df_no_null_subset = df_no_duplicates.dropna(subset=['category', 'address'])
df_no_null_subset.isna().sum()

title                   0
category                0
address                 0
open_hours              0
review_count            0
review_rating           0
reviews_per_rating      0
latitude                0
longitude               0
user_reviews          119
dtype: int64

In [15]:
df_no_null_subset['user_reviews'].fillna(value='-', inplace=True)
df_no_null_subset.isna().sum()

<ipython-input-15-4380ff826ea6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_null_subset['user_reviews'].fillna(value='-', inplace=True)


title                 0
category              0
address               0
open_hours            0
review_count          0
review_rating         0
reviews_per_rating    0
latitude              0
longitude             0
user_reviews          0
dtype: int64

In [16]:
df_no_null_subset

,title,category,address,open_hours,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews
0,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...","{""Friday"":[""7:30 AM–9 PM""],""Monday"":[""7:30 AM–...",95,4.900.000,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155.502,119.440.242,"[{""Name"":""Ade Lhinglhung"",""ProfilePicture"":""ht..."
1,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...","{""Friday"":[""7 AM–9 PM""],""Monday"":[""7 AM–9 PM""]...",9,4.600.000,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161.116,119.413.019,"[{""Name"":""Refly Nakcoc"",""ProfilePicture"":""http..."
2,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...","{""Friday"":[""7 AM–9 PM""],""Monday"":[""7 AM–9 PM""]...",140,4.300.000,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141.826,119.489.119,"[{""Name"":""Triwiyanti Triwiyanti"",""ProfilePictu..."
3,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...","{""Friday"":[""7 AM–12 PM"",""1–10 PM""],""Monday"":[""...",598,4.900.000,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166.727,119.449.074,"[{""Name"":""isra stimiyapmi"",""ProfilePicture"":""h..."
4,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...","{""Friday"":[""8 AM–11 PM""],""Monday"":[""8 AM–11 PM...",75,4.500.000,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131.841,119.499.985,"[{""Name"":""andi adinda"",""ProfilePicture"":""https..."
...,...,...,...,...,...,...,...,...,...,...
808,Laundry,Laundry,"HPG3+W8G Laundry, Jl. Arief Rahman Hakim, Suka...",{},0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.577.300,98.703.334,-
809,Laundry dry & clean,Laundry,"Jl. Jamin Ginting No.96, Mangga, Kec. Medan Tu...","{""Friday"":[""8 AM–10 PM""],""Monday"":[""8 AM–10 PM...",0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.528.993,98.637.205,-
810,Michelle Laundry & Dry Clean,Laundry,"JM8X+532 Michelle Laundry & Dry Clean, Jl. Tua...","{""Friday"":[""8 AM–10 PM""],""Monday"":[""8 AM–10 PM...",1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615.383,98.697.644,"[{""Name"":""Eben RHS"",""ProfilePicture"":""https://..."
811,Laundry,Janitorial service,"HPV6+HX6 Laundry, Gg. Permata, Bantan, Kec. Me...",{},1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.593.911,98.712.397,"[{""Name"":""Aditya Masni"",""ProfilePicture"":""http..."


In [17]:
df_clean = df_no_null_subset
# df_clean.dtypes
# df_clean['open_hours'] = df_clean['open_hours'].apply(ast.literal_eval)

In [18]:
# def clean_columns(df, columns_to_clean):
#     for column_name in columns_to_clean:
#         if column_name in df.columns:
#             df[column_name] = df[column_name].apply(lambda x: {day: [time.replace('\u202f', '') for time in times] for day, times in x.items()})

# clean_columns(df_clean, ['open_hours'])

In [19]:
# print(df_clean['open_hours'])

In [20]:
# def format_open_hours(open_hours_dict):
#     formatted_hours = []
#     for day, hours in open_hours_dict.items():
#         formatted_hours.append(f"{day} {', '.join(hours)}")
#     return ','.join(formatted_hours)

# df_clean['open_hours'] = df_clean['open_hours'].apply(format_open_hours)

In [21]:
# df_clean['open_hours']

In [22]:
df1 = df_clean[['title','category','address','review_count','review_rating','reviews_per_rating','latitude','longitude','user_reviews']]
df1

,title,category,address,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews
0,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...",95,4.900.000,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155.502,119.440.242,"[{""Name"":""Ade Lhinglhung"",""ProfilePicture"":""ht..."
1,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...",9,4.600.000,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161.116,119.413.019,"[{""Name"":""Refly Nakcoc"",""ProfilePicture"":""http..."
2,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...",140,4.300.000,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141.826,119.489.119,"[{""Name"":""Triwiyanti Triwiyanti"",""ProfilePictu..."
3,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...",598,4.900.000,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166.727,119.449.074,"[{""Name"":""isra stimiyapmi"",""ProfilePicture"":""h..."
4,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...",75,4.500.000,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131.841,119.499.985,"[{""Name"":""andi adinda"",""ProfilePicture"":""https..."
...,...,...,...,...,...,...,...,...,...
808,Laundry,Laundry,"HPG3+W8G Laundry, Jl. Arief Rahman Hakim, Suka...",0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.577.300,98.703.334,-
809,Laundry dry & clean,Laundry,"Jl. Jamin Ginting No.96, Mangga, Kec. Medan Tu...",0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.528.993,98.637.205,-
810,Michelle Laundry & Dry Clean,Laundry,"JM8X+532 Michelle Laundry & Dry Clean, Jl. Tua...",1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615.383,98.697.644,"[{""Name"":""Eben RHS"",""ProfilePicture"":""https://..."
811,Laundry,Janitorial service,"HPV6+HX6 Laundry, Gg. Permata, Bantan, Kec. Me...",1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.593.911,98.712.397,"[{""Name"":""Aditya Masni"",""ProfilePicture"":""http..."


In [23]:
df1[['user_reviews']]

,user_reviews
0,"[{""Name"":""Ade Lhinglhung"",""ProfilePicture"":""ht..."
1,"[{""Name"":""Refly Nakcoc"",""ProfilePicture"":""http..."
2,"[{""Name"":""Triwiyanti Triwiyanti"",""ProfilePictu..."
3,"[{""Name"":""isra stimiyapmi"",""ProfilePicture"":""h..."
4,"[{""Name"":""andi adinda"",""ProfilePicture"":""https..."
...,...
808,-
809,-
810,"[{""Name"":""Eben RHS"",""ProfilePicture"":""https://..."
811,"[{""Name"":""Aditya Masni"",""ProfilePicture"":""http..."


In [24]:
# Apply the safe_eval function to the 'user_reviews' column
df1['user_reviews'] = df1['user_reviews'].replace('-', '{}')

<ipython-input-24-011b2386475d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['user_reviews'] = df1['user_reviews'].replace('-', '{}')


In [25]:
def safe_loads(x):
    try:
        return json.loads(x)
    except (json.JSONDecodeError, TypeError):
        return None

# Apply the safe_loads function to the 'user_reviews' column
df1['user_reviews'] = df1['user_reviews'].apply(safe_loads)

<ipython-input-25-9a65978117ad>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['user_reviews'] = df1['user_reviews'].apply(safe_loads)


In [26]:
df1['user_reviews'][0]

[{'Name': 'Ade Lhinglhung',
  'ProfilePicture': 'https://lh3.googleusercontent.com/a-/ALV-UjUu-q9jwXdKQQNQqj8gryJe1PF195ZPnOUkJU9hpFg1_A=s120-c-rp-mo-ba3-br100',
  'Rating': 5,
  'Description': 'Pertama kalinya loundry disini, bawa Baju putih yang bernoda kl loundry tempat sebelumnya itu campur, bahkan ada 1 yang bernoda sdh tahunan dan dibiarkan saja. Bawa ke sini hasilnya kembali BERSIH mana rapi lagi hasilnya trus harganya masih terjangkau …',
  'Images': ['https://lh5.googleusercontent.com/p/AF1QipPVmTNjh5BMTZdgsDSIBwYhD-_m7ysbBgL0VR9x=w150-h150-k-no-p',
   'https://lh5.googleusercontent.com/p/AF1QipMroXmDppALluu1KbqxI7qW9SOxd8yh6wPfFxpH=w150-h150-k-no-p'],
  'When': '3 months ago'},
 {'Name': 'Lusiana Duhay',
  'ProfilePicture': 'https://lh3.googleusercontent.com/a/ACg8ocLN6LBOrRO-9dVIl3xVflTLfttauId_DFNat4nCoGwc=s120-c-rp-mo-br100',
  'Rating': 5,
  'Description': 'Happy jd langganan di Lavanderia Laundry. Pelayanan baik dan resposive. Hasil cucian bersih dan wangi. Terakhir sy c

In [27]:

def get_desc(data):
  descriptions_list = [entry.get('Description', '') for entry in data]
  descriptions_sentences = ', '.join(filter(None, descriptions_list))
  return descriptions_sentences

# def get_desc(review):
#   temp = []
#   list_of_desc = []
#   sentences = ''
#   for dic in review:
#     for key,value in dic.items():
#       if key == 'Description':
#         temp.append(value)
#     list_of_desc.append(', '.join(temp))
#   return list_of_desc


df1['user_reviews'] = df1['user_reviews'].apply(get_desc)

<ipython-input-27-359c8c776857>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['user_reviews'] = df1['user_reviews'].apply(get_desc)


In [28]:
df1['user_reviews'][0]

'Pertama kalinya loundry disini, bawa Baju putih yang bernoda kl loundry tempat sebelumnya itu campur, bahkan ada 1 yang bernoda sdh tahunan dan dibiarkan saja. Bawa ke sini hasilnya kembali BERSIH mana rapi lagi hasilnya trus harganya masih terjangkau …, Happy jd langganan di Lavanderia Laundry. Pelayanan baik dan resposive. Hasil cucian bersih dan wangi. Terakhir sy cuci bantal Tempur yg byk noda dan jamur. Ternyataa bantal jd bersihhh donk, happy sekali. Recommended laundry disini., Pelayanan cepat. Mohon ditambahkan jadwal operasional di lembar kuitansi supaya customer bisa mengetahui batas pengambilan. Terima kasih Lavanderia Laundry, Tempatnya strategis di daerah manakukang. Laundry Antar jemput paling cepet. Hasil cuciannya juga bersih, setrika rapi, wangi sekali., tau ga sih ...pengahsilan dari bisnis ini sebagian d gunakan buat dana sosial lo....jadi klo kita k sini kita juga bisa sambil mendukung kegiatan sosial tersebut...bersih harum...dan berpahala deh....sukses ya Lavande

In [29]:
df1

,title,category,address,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews
0,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...",95,4.900.000,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155.502,119.440.242,"Pertama kalinya loundry disini, bawa Baju puti..."
1,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...",9,4.600.000,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161.116,119.413.019,Tempatnya bersih.... Parfum yg di pake wangi b...
2,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...",140,4.300.000,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141.826,119.489.119,"Sdh menunggu antri selama 1 jam, baru diinfoka..."
3,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...",598,4.900.000,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166.727,119.449.074,TDK BISA BERKATA2 SOAL LAUNDRY INI. SEMUA ULAS...
4,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...",75,4.500.000,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131.841,119.499.985,"Waktu pertama kali kesini saya belum paham, te..."
...,...,...,...,...,...,...,...,...,...
808,Laundry,Laundry,"HPG3+W8G Laundry, Jl. Arief Rahman Hakim, Suka...",0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.577.300,98.703.334,
809,Laundry dry & clean,Laundry,"Jl. Jamin Ginting No.96, Mangga, Kec. Medan Tu...",0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.528.993,98.637.205,
810,Michelle Laundry & Dry Clean,Laundry,"JM8X+532 Michelle Laundry & Dry Clean, Jl. Tua...",1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615.383,98.697.644,Bersih bersih
811,Laundry,Janitorial service,"HPV6+HX6 Laundry, Gg. Permata, Bantan, Kec. Me...",1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.593.911,98.712.397,Hasilnya sangat memuaskan


In [30]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [31]:
df1['user_reviews'] = df1['user_reviews'].apply(stemmer.stem)

<ipython-input-31-39feba2a2159>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['user_reviews'] = df1['user_reviews'].apply(stemmer.stem)


In [32]:
df1['user_reviews'][0]



'pertama kali loundry sini bawa baju putih yang noda kl loundry tempat belum itu campur bahkan ada 1 yang noda sdh tahun dan biar saja bawa ke sini hasil kembali bersih mana rapi lagi hasil trus harga masih jangkau happy jd langgan di lavanderia laundry layan baik dan resposive hasil cuci bersih dan wangi akhir sy cuci bantal tempur yg byk noda dan jamur ternyataa bantal jd bersihhh donk happy sekali recommended laundry sini layan cepat mohon tambah jadwal operasional di lembar kuitansi supaya customer bisa tahu batas ambil terima kasih lavanderia laundry tempat strategis di daerah manakukang laundry antar jemput paling cepet hasil cuci juga bersih setrika rapi wangi sekali tau ga sih pengahsilan dari bisnis ini bagi d guna buat dana sosial lo jadi klo kita k sini kita juga bisa sambil dukung giat sosial sebut bersih harum dan pahala deh sukses ya lavanderia laundry layan bagus bisa antar jemput dan lokasi trjangkau rapih wangi poko nya top banget dehhh lavanderia laundry antar jemput 

In [33]:
stopword_remover = StopWordRemoverFactory().create_stop_word_remover()
def remove_stopwords(text):
    return stopword_remover.remove(text)

df1['user_reviews'] = df1['user_reviews'].apply(remove_stopwords)



<ipython-input-33-bc9e878a4bf2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['user_reviews'] = df1['user_reviews'].apply(remove_stopwords)


In [34]:
# Define your custom stopwords
custom_stopwords = {"yg", "yah", "aja", "disni", "dr", "you", "ga", "gua", "gak", "yaaa", "krn", "gt", "dah", "kalo", "wa", "kl"}

# Function to remove custom stopwords
def remove_custom_stopwords(text):
    words = text.split(' ')
    filtered_words = [word for word in words if word not in custom_stopwords]

    return ' '.join(filtered_words)

# Apply the remove_custom_stopwords function to your DataFrame column
df1['user_reviews'] = df1['user_reviews'].apply(remove_custom_stopwords)
df1

<ipython-input-34-ddcf273e7796>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['user_reviews'] = df1['user_reviews'].apply(remove_custom_stopwords)


,title,category,address,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews
0,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...",95,4.900.000,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155.502,119.440.242,loundry bawa baju putih noda loundry campur 1 ...
1,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...",9,4.600.000,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161.116,119.413.019,bersih parfum pake wangi banget so recomend la...
2,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...",140,4.300.000,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141.826,119.489.119,sdh tunggu antri 1 jam info tdk bs cuci kering...
3,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...",598,4.900.000,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166.727,119.449.074,tdk berkata2 laundry ulas bintang 5 settingan ...
4,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...",75,4.500.000,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131.841,119.499.985,waktu kesini paham 8 kg muat 1 mesin jaga suru...
...,...,...,...,...,...,...,...,...,...
808,Laundry,Laundry,"HPG3+W8G Laundry, Jl. Arief Rahman Hakim, Suka...",0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.577.300,98.703.334,
809,Laundry dry & clean,Laundry,"Jl. Jamin Ginting No.96, Mangga, Kec. Medan Tu...",0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.528.993,98.637.205,
810,Michelle Laundry & Dry Clean,Laundry,"JM8X+532 Michelle Laundry & Dry Clean, Jl. Tua...",1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615.383,98.697.644,bersih bersih
811,Laundry,Janitorial service,"HPV6+HX6 Laundry, Gg. Permata, Bantan, Kec. Me...",1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.593.911,98.712.397,hasil muas


In [35]:
df1['user_reviews'][0]

'loundry bawa baju putih noda loundry campur 1 noda sdh tahun biar bawa hasil bersih rapi hasil trus harga jangkau happy jd langgan lavanderia laundry layan resposive hasil cuci bersih wangi sy cuci bantal tempur byk noda jamur ternyataa bantal jd bersihhh donk happy recommended laundry layan cepat jadwal operasional lembar kuitansi customer batas ambil terima kasih lavanderia laundry strategis daerah manakukang laundry jemput cepet hasil cuci bersih setrika rapi wangi tau sih pengahsilan bisnis dana sosial lo klo dukung giat sosial bersih harum pahala deh sukses lavanderia laundry layan bagus jemput lokasi trjangkau rapih wangi poko top banget dehhh lavanderia laundry jemput cepat hasil bersih rekomended lavanderia laundry cepat bersih banget wangi banget mantap poko laundry info teman kantor bagus laundry wangi sih wangi baju kayak bersih kerah dakianji'

In [36]:
df1

,title,category,address,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews
0,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...",95,4.900.000,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155.502,119.440.242,loundry bawa baju putih noda loundry campur 1 ...
1,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...",9,4.600.000,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161.116,119.413.019,bersih parfum pake wangi banget so recomend la...
2,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...",140,4.300.000,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141.826,119.489.119,sdh tunggu antri 1 jam info tdk bs cuci kering...
3,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...",598,4.900.000,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166.727,119.449.074,tdk berkata2 laundry ulas bintang 5 settingan ...
4,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...",75,4.500.000,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131.841,119.499.985,waktu kesini paham 8 kg muat 1 mesin jaga suru...
...,...,...,...,...,...,...,...,...,...
808,Laundry,Laundry,"HPG3+W8G Laundry, Jl. Arief Rahman Hakim, Suka...",0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.577.300,98.703.334,
809,Laundry dry & clean,Laundry,"Jl. Jamin Ginting No.96, Mangga, Kec. Medan Tu...",0,0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.528.993,98.637.205,
810,Michelle Laundry & Dry Clean,Laundry,"JM8X+532 Michelle Laundry & Dry Clean, Jl. Tua...",1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615.383,98.697.644,bersih bersih
811,Laundry,Janitorial service,"HPV6+HX6 Laundry, Gg. Permata, Bantan, Kec. Me...",1,5.000.000,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.593.911,98.712.397,hasil muas


In [37]:
# Corrected code for casting data types
df1[['title', 'category', 'address']] = df1[['title', 'category', 'address']].astype(str)
df1['review_rating'] = df1['review_rating'].astype(str)

<ipython-input-37-571ef0f4f79f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[['title', 'category', 'address']] = df1[['title', 'category', 'address']].astype(str)
<ipython-input-37-571ef0f4f79f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['review_rating'] = df1['review_rating'].astype(str)


In [38]:
df1['review_rating'] = df1['review_rating'].apply(lambda x: float(x[:3]))

<ipython-input-38-041ef654131d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['review_rating'] = df1['review_rating'].apply(lambda x: float(x[:3]))


In [39]:
df1['review_rating']

0      4.9
1      4.6
2      4.3
3      4.9
4      4.5
      ... 
808    0.0
809    0.0
810    5.0
811    5.0
812    0.0
Name: review_rating, Length: 790, dtype: float64

In [40]:
# df1[['review_count','review_rating','latitude','longitude']] = df1[['review_count','review_rating','latitude','longitude']].astype(float)
latitude = df1['latitude']
longitude = df1['longitude']

In [41]:
latitude = latitude.apply(lambda x: x.rsplit('.', 1)[0] + x.rsplit('.', 1)[1] if x.count('.') > 1 else x)
longitude = longitude.apply(lambda x: x.rsplit('.', 1)[0] + x.rsplit('.', 1)[1] if x.count('.') > 1 else x)

In [42]:
df1['latitude'] = latitude.astype('float32')
df1['longitude'] = longitude.astype('float32')

<ipython-input-42-4d47eaade05f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['latitude'] = latitude.astype('float32')
<ipython-input-42-4d47eaade05f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['longitude'] = longitude.astype('float32')


In [43]:
df1['review_count'] = df1['review_count'].astype(int)

<ipython-input-43-d67a5e00830f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['review_count'] = df1['review_count'].astype(int)


In [44]:
df1

,title,category,address,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews
0,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...",95,4.9,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155502,119.440239,loundry bawa baju putih noda loundry campur 1 ...
1,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...",9,4.6,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161116,119.413017,bersih parfum pake wangi banget so recomend la...
2,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...",140,4.3,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141826,119.489120,sdh tunggu antri 1 jam info tdk bs cuci kering...
3,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...",598,4.9,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166727,119.449074,tdk berkata2 laundry ulas bintang 5 settingan ...
4,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...",75,4.5,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131841,119.499985,waktu kesini paham 8 kg muat 1 mesin jaga suru...
...,...,...,...,...,...,...,...,...,...
808,Laundry,Laundry,"HPG3+W8G Laundry, Jl. Arief Rahman Hakim, Suka...",0,0.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.577300,98.703331,
809,Laundry dry & clean,Laundry,"Jl. Jamin Ginting No.96, Mangga, Kec. Medan Tu...",0,0.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.528993,98.637207,
810,Michelle Laundry & Dry Clean,Laundry,"JM8X+532 Michelle Laundry & Dry Clean, Jl. Tua...",1,5.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615383,98.697647,bersih bersih
811,Laundry,Janitorial service,"HPV6+HX6 Laundry, Gg. Permata, Bantan, Kec. Me...",1,5.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.593911,98.712395,hasil muas


In [45]:
df1.dtypes

title                  object
category               object
address                object
review_count            int64
review_rating         float64
reviews_per_rating     object
latitude              float32
longitude             float32
user_reviews           object
dtype: object

In [46]:
df1.columns

Index(['title', 'category', 'address', 'review_count', 'review_rating',
       'reviews_per_rating', 'latitude', 'longitude', 'user_reviews'],
      dtype='object')

In [47]:
df1['user_reviews'] = df1['user_reviews'].fillna('')
df1

<ipython-input-47-0c8f0a5fc34a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['user_reviews'] = df1['user_reviews'].fillna('')


,title,category,address,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews
0,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...",95,4.9,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155502,119.440239,loundry bawa baju putih noda loundry campur 1 ...
1,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...",9,4.6,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161116,119.413017,bersih parfum pake wangi banget so recomend la...
2,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...",140,4.3,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141826,119.489120,sdh tunggu antri 1 jam info tdk bs cuci kering...
3,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...",598,4.9,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166727,119.449074,tdk berkata2 laundry ulas bintang 5 settingan ...
4,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...",75,4.5,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131841,119.499985,waktu kesini paham 8 kg muat 1 mesin jaga suru...
...,...,...,...,...,...,...,...,...,...
808,Laundry,Laundry,"HPG3+W8G Laundry, Jl. Arief Rahman Hakim, Suka...",0,0.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.577300,98.703331,
809,Laundry dry & clean,Laundry,"Jl. Jamin Ginting No.96, Mangga, Kec. Medan Tu...",0,0.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.528993,98.637207,
810,Michelle Laundry & Dry Clean,Laundry,"JM8X+532 Michelle Laundry & Dry Clean, Jl. Tua...",1,5.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615383,98.697647,bersih bersih
811,Laundry,Janitorial service,"HPV6+HX6 Laundry, Gg. Permata, Bantan, Kec. Me...",1,5.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.593911,98.712395,hasil muas


In [48]:
# Tokenize and pad the input text

df1['latitude'] = df1['latitude'].astype('float32')
df1['longitude'] = df1['longitude'].astype('float32')
df1['review_count'] = df1['review_count'].astype('float32')
df1['review_rating'] = df1['review_rating'].astype('float32')
df1

<ipython-input-48-41681403e67e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['latitude'] = df1['latitude'].astype('float32')
<ipython-input-48-41681403e67e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['longitude'] = df1['longitude'].astype('float32')
<ipython-input-48-41681403e67e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

,title,category,address,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews
0,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...",95.0,4.9,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155502,119.440239,loundry bawa baju putih noda loundry campur 1 ...
1,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...",9.0,4.6,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161116,119.413017,bersih parfum pake wangi banget so recomend la...
2,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...",140.0,4.3,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141826,119.489120,sdh tunggu antri 1 jam info tdk bs cuci kering...
3,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...",598.0,4.9,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166727,119.449074,tdk berkata2 laundry ulas bintang 5 settingan ...
4,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...",75.0,4.5,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131841,119.499985,waktu kesini paham 8 kg muat 1 mesin jaga suru...
...,...,...,...,...,...,...,...,...,...
808,Laundry,Laundry,"HPG3+W8G Laundry, Jl. Arief Rahman Hakim, Suka...",0.0,0.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.577300,98.703331,
809,Laundry dry & clean,Laundry,"Jl. Jamin Ginting No.96, Mangga, Kec. Medan Tu...",0.0,0.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":0}",3.528993,98.637207,
810,Michelle Laundry & Dry Clean,Laundry,"JM8X+532 Michelle Laundry & Dry Clean, Jl. Tua...",1.0,5.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615383,98.697647,bersih bersih
811,Laundry,Janitorial service,"HPV6+HX6 Laundry, Gg. Permata, Bantan, Kec. Me...",1.0,5.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.593911,98.712395,hasil muas


In [49]:
def get_top_words(text, n=10):
    vectorizer = CountVectorizer(stop_words=stop_words)
    X = vectorizer.fit_transform([text])
    words = vectorizer.get_feature_names_out()
    word_counts = X.toarray().flatten()
    top_indices = word_counts.argsort()[-n:][::-1]
    top_words = [words[i] for i in top_indices]
    return ' '.join(top_words)

df1 = df1[df1['user_reviews'] != '']
stop_words = ['']
# Reset the index after dropping rows
df1.reset_index(drop=True, inplace=True)
df1['address'] = df1['address'].str.split(',').str[1:].str.join(',').str.strip()
df1['address'] = df1['address'].str[:-6].str.strip()
df1['address'] = df1['address'].str.rsplit(', ', 3).apply(lambda x: ', '.join(x[-3:])).str.strip()
df1['address'] = df1['address'].apply(lambda x: x.replace('Kec. ', '') if 'Kec. ' in x else x)
df1['top_words'] = df1['user_reviews'].apply(get_top_words)

df1

<ipython-input-49-8c2cf6ef175b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['address'] = df1['address'].str.split(',').str[1:].str.join(',').str.strip()
<ipython-input-49-8c2cf6ef175b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['address'] = df1['address'].str[:-6].str.strip()
<ipython-input-49-8c2cf6ef175b>:16: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df1['address'] = df1['address'].str.rspli

,title,category,address,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews,top_words
0,Lavanderia,Laundry service,"Panakkukang, Makassar City, South Sulawesi",95.0,4.9,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155502,119.440239,loundry bawa baju putih noda loundry campur 1 ...,laundry bersih wangi hasil lavanderia cepat ba...
1,LaundrySaja,Laundry service,"Mariso, Makassar City, South Sulawesi",9.0,4.6,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161116,119.413017,bersih parfum pake wangi banget so recomend la...,laundry terima dgn service bersih pegawai laya...
2,Zap Laundry,Laundry service,"Tamalanrea, Kota Makassar, Sulawesi Selatan",140.0,4.3,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141826,119.489120,sdh tunggu antri 1 jam info tdk bs cuci kering...,cuci mesin service jam self layan kering kesin...
3,QnC Laundry Toddopuli,Laundry service,"Rappocini, Makassar City, South Sulawesi",598.0,4.9,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166727,119.449074,tdk berkata2 laundry ulas bintang 5 settingan ...,layan laundry hasil qnc bersih muas wangi rama...
4,Laundromad Makassar,Laundry service,"Tamalanrea, Makassar City, South Sulawesi",75.0,4.5,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131841,119.499985,waktu kesini paham 8 kg muat 1 mesin jaga suru...,antri kesini nomor mesin bagus tunggu jaga cuc...
...,...,...,...,...,...,...,...,...,...,...
602,Z-Laundry,Laundry service,"Medan Kota, Kota Medan, Sumatera Utara",4.0,2.5,"{""1"":2,""2"":0,""3"":1,""4"":0,""5"":1}",3.563773,98.696487,mayan bersih puas,puas mayan bersih
603,Tiwa Laundry,Clothing store,"Medan Selayang, Kota Medan, Sumatera Utara",6.0,3.7,"{""1"":1,""2"":0,""3"":2,""4"":0,""5"":3}",3.558922,98.648346,laundry mahal bersih cuci ulang udah cuci boho...,laundry cuci ulang udah tiwa mahal jaya buruk ...
604,DeLaundress Dry Cleaning & Laundry,Dry cleaner,"Medan Kota, Medan City, North Sumatra",2.0,3.5,"{""1"":0,""2"":0,""3"":1,""4"":1,""5"":0}",3.583483,98.691444,clean and modern place,place modern clean and
605,Michelle Laundry & Dry Clean,Laundry,"Medan Tembung, Kota Medan, Sumatera Utara",1.0,5.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615383,98.697647,bersih bersih,bersih


In [50]:
def reviews_to_one_hot(row):
    ratings = json.loads(row)
    one_hot = {'{}_star'.format(key): ratings[key] for key in ratings}
    return pd.Series(one_hot)

df_one_hot = df1['reviews_per_rating'].apply(reviews_to_one_hot)

df1 = pd.concat([df1, df_one_hot], axis=1)
df1

,title,category,address,review_count,review_rating,reviews_per_rating,latitude,longitude,user_reviews,top_words,1_star,2_star,3_star,4_star,5_star
0,Lavanderia,Laundry service,"Panakkukang, Makassar City, South Sulawesi",95.0,4.9,"{""1"":1,""2"":0,""3"":1,""4"":2,""5"":91}",-5.155502,119.440239,loundry bawa baju putih noda loundry campur 1 ...,laundry bersih wangi hasil lavanderia cepat ba...,1,0,1,2,91
1,LaundrySaja,Laundry service,"Mariso, Makassar City, South Sulawesi",9.0,4.6,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":8}",-5.161116,119.413017,bersih parfum pake wangi banget so recomend la...,laundry terima dgn service bersih pegawai laya...,1,0,0,0,8
2,Zap Laundry,Laundry service,"Tamalanrea, Kota Makassar, Sulawesi Selatan",140.0,4.3,"{""1"":13,""2"":0,""3"":7,""4"":30,""5"":90}",-5.141826,119.489120,sdh tunggu antri 1 jam info tdk bs cuci kering...,cuci mesin service jam self layan kering kesin...,13,0,7,30,90
3,QnC Laundry Toddopuli,Laundry service,"Rappocini, Makassar City, South Sulawesi",598.0,4.9,"{""1"":9,""2"":1,""3"":2,""4"":31,""5"":555}",-5.166727,119.449074,tdk berkata2 laundry ulas bintang 5 settingan ...,layan laundry hasil qnc bersih muas wangi rama...,9,1,2,31,555
4,Laundromad Makassar,Laundry service,"Tamalanrea, Makassar City, South Sulawesi",75.0,4.5,"{""1"":4,""2"":3,""3"":2,""4"":11,""5"":55}",-5.131841,119.499985,waktu kesini paham 8 kg muat 1 mesin jaga suru...,antri kesini nomor mesin bagus tunggu jaga cuc...,4,3,2,11,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,Z-Laundry,Laundry service,"Medan Kota, Kota Medan, Sumatera Utara",4.0,2.5,"{""1"":2,""2"":0,""3"":1,""4"":0,""5"":1}",3.563773,98.696487,mayan bersih puas,puas mayan bersih,2,0,1,0,1
603,Tiwa Laundry,Clothing store,"Medan Selayang, Kota Medan, Sumatera Utara",6.0,3.7,"{""1"":1,""2"":0,""3"":2,""4"":0,""5"":3}",3.558922,98.648346,laundry mahal bersih cuci ulang udah cuci boho...,laundry cuci ulang udah tiwa mahal jaya buruk ...,1,0,2,0,3
604,DeLaundress Dry Cleaning & Laundry,Dry cleaner,"Medan Kota, Medan City, North Sumatra",2.0,3.5,"{""1"":0,""2"":0,""3"":1,""4"":1,""5"":0}",3.583483,98.691444,clean and modern place,place modern clean and,0,0,1,1,0
605,Michelle Laundry & Dry Clean,Laundry,"Medan Tembung, Kota Medan, Sumatera Utara",1.0,5.0,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",3.615383,98.697647,bersih bersih,bersih,0,0,0,0,1


In [51]:
dff = df1[['title','category','address','review_count','review_rating','1_star','2_star','3_star','4_star','5_star','latitude','longitude','top_words',]]
dff

,title,category,address,review_count,review_rating,1_star,2_star,3_star,4_star,5_star,latitude,longitude,top_words
0,Lavanderia,Laundry service,"Panakkukang, Makassar City, South Sulawesi",95.0,4.9,1,0,1,2,91,-5.155502,119.440239,laundry bersih wangi hasil lavanderia cepat ba...
1,LaundrySaja,Laundry service,"Mariso, Makassar City, South Sulawesi",9.0,4.6,1,0,0,0,8,-5.161116,119.413017,laundry terima dgn service bersih pegawai laya...
2,Zap Laundry,Laundry service,"Tamalanrea, Kota Makassar, Sulawesi Selatan",140.0,4.3,13,0,7,30,90,-5.141826,119.489120,cuci mesin service jam self layan kering kesin...
3,QnC Laundry Toddopuli,Laundry service,"Rappocini, Makassar City, South Sulawesi",598.0,4.9,9,1,2,31,555,-5.166727,119.449074,layan laundry hasil qnc bersih muas wangi rama...
4,Laundromad Makassar,Laundry service,"Tamalanrea, Makassar City, South Sulawesi",75.0,4.5,4,3,2,11,55,-5.131841,119.499985,antri kesini nomor mesin bagus tunggu jaga cuc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,Z-Laundry,Laundry service,"Medan Kota, Kota Medan, Sumatera Utara",4.0,2.5,2,0,1,0,1,3.563773,98.696487,puas mayan bersih
603,Tiwa Laundry,Clothing store,"Medan Selayang, Kota Medan, Sumatera Utara",6.0,3.7,1,0,2,0,3,3.558922,98.648346,laundry cuci ulang udah tiwa mahal jaya buruk ...
604,DeLaundress Dry Cleaning & Laundry,Dry cleaner,"Medan Kota, Medan City, North Sumatra",2.0,3.5,0,0,1,1,0,3.583483,98.691444,place modern clean and
605,Michelle Laundry & Dry Clean,Laundry,"Medan Tembung, Kota Medan, Sumatera Utara",1.0,5.0,0,0,0,0,1,3.615383,98.697647,bersih


In [52]:
title_le = LabelEncoder()
dff['encoded_title'] = title_le.fit_transform(dff['title'])
# Tokenize and pad the input text
tokenizer_reviews = tf.keras.preprocessing.text.Tokenizer()
tokenizer_reviews.fit_on_texts(dff['top_words'])
vocab_size_reviews = len(tokenizer_reviews.word_index) + 1

max_length_reviews = 10
X_train_sequences_reviews = tokenizer_reviews.texts_to_sequences(dff['top_words'])
X_train_padded_reviews = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences_reviews, maxlen=max_length_reviews, padding='post')

<ipython-input-52-aef97bbfd6f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['encoded_title'] = title_le.fit_transform(dff['title'])


In [53]:
# Tokenize and pad the address
tokenizer_address = tf.keras.preprocessing.text.Tokenizer()
tokenizer_address.fit_on_texts(dff['address'])
vocab_size_address = len(tokenizer_address.word_index) + 1

max_length_address = 10
X_train_sequences_address = tokenizer_address.texts_to_sequences(dff['address'])
X_train_padded_address = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences_address, maxlen=max_length_address, padding='post')

In [54]:
# Normalize numerical features
scaler = StandardScaler()
dff[['review_count', 'review_rating']] = scaler.fit_transform(dff[['review_count', 'review_rating']])

<ipython-input-54-048e8c70df11>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff[['review_count', 'review_rating']] = scaler.fit_transform(dff[['review_count', 'review_rating']])


In [55]:
label_encoders = {}
categorical_columns = ['category']
for column in categorical_columns:
    le = LabelEncoder()
    dff[column] = le.fit_transform(dff[column])
    label_encoders[column] = le

<ipython-input-55-7dfc790d3333>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff[column] = le.fit_transform(dff[column])


In [56]:
embedding_dim = 100

category_input = Input(shape=(1,))
address_input = Input(shape=(max_length_address,))
review_count_input = Input(shape=(1,))
review_rating_input = Input(shape=(1,))
latitude_input = Input(shape=(1,))
longitude_input = Input(shape=(1,))
stars_input = Input(shape=(5,))

category_embedding = Embedding(input_dim=len(dff['category'].unique()), output_dim=embedding_dim)(category_input)
address_embedding = Embedding(input_dim=vocab_size_address, output_dim=embedding_dim, input_length=max_length_address)(address_input)

In [57]:
# Flatten the embeddings
category_flat = tf.keras.layers.Flatten()(category_embedding)
address_flat = tf.keras.layers.Flatten()(address_embedding)

In [58]:
# Concatenate all embeddings
all_embeddings = Concatenate()([category_flat, address_flat, review_count_input, review_rating_input, latitude_input, longitude_input,stars_input])

In [59]:
# Create an embedding layer for the text
text_input = Input(shape=(max_length_reviews,))
text_embedding = Embedding(input_dim=vocab_size_reviews, output_dim=embedding_dim, input_length=max_length_reviews)(text_input)
text_avg_embedding = GlobalAveragePooling1D()(text_embedding)

In [60]:
# Concatenate the text embedding with other embeddings
all_embeddings = Concatenate()([all_embeddings, text_avg_embedding])

In [61]:
# Fully connected layers for final prediction
dense_layer = Dense(128, activation='relu')(all_embeddings)
dense_layer = Dense(64,activation='relu')(dense_layer)
output = Dense(len(set(dff['encoded_title'])), activation='softmax')(dense_layer)

In [62]:
# Create the model
model = Model(inputs=[category_input, address_input, review_count_input, review_rating_input, latitude_input, longitude_input,stars_input, text_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [63]:
model.fit(
    [dff['category'], X_train_padded_address, dff['review_count'], dff['review_rating'], dff['latitude'], dff['longitude'],np.array(dff[['1_star', '2_star', '3_star', '4_star', '5_star']]), X_train_padded_reviews],
    dff['encoded_title'], epochs=100, batch_size=64
)

Epoch 1/100
10/10 [==============================] - 3s 18ms/step - loss: 6.7185 - accuracy: 0.0016
Epoch 2/100
10/10 [==============================] - 0s 23ms/step - loss: 6.3753 - accuracy: 0.0016
Epoch 3/100
10/10 [==============================] - 0s 15ms/step - loss: 6.3247 - accuracy: 0.0066
Epoch 4/100
10/10 [==============================] - 0s 19ms/step - loss: 6.2771 - accuracy: 0.0049
Epoch 5/100
10/10 [==============================] - 0s 17ms/step - loss: 6.2109 - accuracy: 0.0165
Epoch 6/100
10/10 [==============================] - 0s 16ms/step - loss: 6.1233 - accuracy: 0.0082
Epoch 7/100
10/10 [==============================] - 0s 16ms/step - loss: 6.0322 - accuracy: 0.0132
Epoch 8/100
10/10 [==============================] - 0s 15ms/step - loss: 5.9297 - accuracy: 0.0231
Epoch 9/100
10/10 [==============================] - 0s 16ms/step - loss: 5.8238 - accuracy: 0.0165
Epoch 10/100
10/10 [==============================] - 0s 20ms/step - loss: 5.7084 - accuracy: 0.0280

In [64]:
# Assuming you have new data for prediction
new_data = {
    'title': ["LaundrySaja"],
    'category': ["Dry cleaner"],
    'address': ["Mariso, Makassar City, South Sulawesi"],
    'review_count': [9.0],
    'review_rating': [4.599999904632568],
    'latitude': [-5.161116123199463],
    'longitude': [119.41301727294922],
    'user_reviews': ["wangi bersih"]
}

# Convert the new data to a DataFrame
new_df = pd.DataFrame(new_data)

# Tokenize and pad the input text for the new data
X_new_sequences_reviews = tokenizer_reviews.texts_to_sequences(new_df['user_reviews'])
X_new_padded_reviews = tf.keras.preprocessing.sequence.pad_sequences(X_new_sequences_reviews, maxlen=max_length_reviews, padding='post')

X_new_sequences_address = tokenizer_address.texts_to_sequences(new_df['address'])
X_new_padded_address = tf.keras.preprocessing.sequence.pad_sequences(X_new_sequences_address, maxlen=max_length_address, padding='post')

# Assume you have categorical data for the category
new_df['category'] = label_encoders['category'].transform(new_df['category'])

# Make predictions for the new data
predictions = model.predict([
    [np.array(new_df['category'])], [X_new_padded_address],
    [np.array(new_df['review_count'])], [np.array(new_df['review_rating'])],
    [np.array(new_df['latitude'])], [np.array(new_df['longitude'])], [X_new_padded_reviews]
])


# Get the indices of the top 5 predictions
top_n_indices = np.argsort(predictions, axis=1)[:, -10:]

# Initialize a list to store the top 5 predictions with their features
top_n_predictions_with_features = []

# Loop through each set of indices
for indices in top_n_indices:
    # Decode the predictions using the inverse_transform method
    decoded_titles = title_le.inverse_transform(indices)

    # Get the corresponding features for each decoded title
    predictions_with_features = []
    for title in decoded_titles:
        index = df1[df1['title'] == title].index[0]  # Get the index of the matching title
        features = df1.iloc[index].to_dict()  # Get all features for that index
        predictions_with_features.append(features)

    top_n_predictions_with_features.append(predictions_with_features)

# Display the top 5 predictions with their features
for i, predictions_with_features in enumerate(top_n_predictions_with_features):
    print("Top 5 Predictions with Features for Input #{}:".format(i + 1))
    for prediction in predictions_with_features:
        print(prediction)
    print("\n")



ValueError: ignored

In [ ]:
# Assuming you have new data for prediction
new_data = {
    'title': ["LaundrySaja"],
    'category': ["Dry cleaner"],
    'address': ["Mariso, Makassar City, South Sulawesi 90122"],
    'review_count': [9.0	],
    'review_rating': [4.599999904632568],
    'latitude': [-5.161116123199463],
    'longitude': [119.41301727294922],
    'user_reviews': ["bersih parfum yg pake wangi banget so recomend yg layan cepat untk org kantor strongly recommend this laundry shop affordable - rp 6000 kg for washing folding service fast service - laundry done within 3 hrs meticulous staff great service if you are in makassar and need laundry services you can consider this shop kecewa dgn layan utama pegawaix yg terima dgn complain dr cuatomerx sarung bantal hilang dgn alas dr pegawai krn hrg promo mrk kewalahan baju bayi milik customer barang yg laundry bantal boneka terima dlm basah alasanx sdh msk mesin ering mgkn pegawai ecek kmbl komplain tp pegawai welcome next laundry lg dsni sdm yg tdk bagus mgkn training dlu utk sopan santun terima kasih cocok langgan laundry effisien murah jaga ramah bersih pakai bersih wangi very recommended"]
}

# Convert the new data to a DataFrame
new_df = pd.DataFrame(new_data)

# Tokenize and pad the input text for the new data
X_new_sequences_reviews = tokenizer_reviews.texts_to_sequences(new_df['user_reviews'])
X_new_padded_reviews = tf.keras.preprocessing.sequence.pad_sequences(X_new_sequences_reviews, maxlen=max_length_reviews, padding='post')

X_new_sequences_title = tokenizer_title.texts_to_sequences(new_df['title'])
X_new_padded_title = tf.keras.preprocessing.sequence.pad_sequences(X_new_sequences_title, maxlen=max_length_title, padding='post')

X_new_sequences_address = tokenizer_address.texts_to_sequences(new_df['address'])
X_new_padded_address = tf.keras.preprocessing.sequence.pad_sequences(X_new_sequences_address, maxlen=max_length_address, padding='post')

# Assume you have categorical data for the category
new_df['category'] = label_encoders['category'].transform(new_df['category'])

# Make predictions for the new data
predictions = model.predict([
    [X_new_padded_title], [np.array(new_df['category'])], [X_new_padded_address],
    [np.array(new_df['review_count'])], [np.array(new_df['review_rating'])],
    [np.array(new_df['latitude'])], [np.array(new_df['longitude'])], [X_new_padded_reviews]
])


# Get the indices of the top 5 predictions
top_n_indices = np.argsort(predictions, axis=1)[:, -10:]

# Initialize a list to store the top 5 predictions with their features
top_n_predictions_with_features = []

# Loop through each set of indices
for indices in top_n_indices:
    # Decode the predictions using the inverse_transform method
    decoded_titles = title_le.inverse_transform(indices)

    # Get the corresponding features for each decoded title
    predictions_with_features = []
    for title in decoded_titles:
        index = df1[df1['title'] == title].index[0]  # Get the index of the matching title
        features = df1.iloc[index].to_dict()  # Get all features for that index
        predictions_with_features.append(features)

    top_n_predictions_with_features.append(predictions_with_features)

# Display the top 5 predictions with their features
for i, predictions_with_features in enumerate(top_n_predictions_with_features):
    print("Top 5 Predictions with Features for Input #{}:".format(i + 1))
    for prediction in predictions_with_features:
        print(prediction)
    print("\n")

